In [1]:
from paddlenlp.datasets import load_dataset
from paddlenlp.data import Tuple, Pad
from functools import partial
import paddle
paddle.set_device("cpu")
from paddle.io import DataLoader
from paddlenlp.transformers import T5Tokenizer as PDT5Tokenizer
from transformers.models.t5 import T5Tokenizer as HGT5Tokenizer
pd_tokenizer = PDT5Tokenizer.from_pretrained("t5-base")
hg_tokenizer = HGT5Tokenizer.from_pretrained("t5-small")

train_ds,dev_ds = load_dataset("glue", "rte", splits=["train","dev"])
id2label = dict(zip(range(len(train_ds.label_list)),train_ds.label_list))

C:\Users\yujun\AppData\Local\Programs\Python\Python37\lib\site-packages\urllib3\util\selectors.py:14: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import namedtuple, Mapping
C:\Users\yujun\AppData\Local\Programs\Python\Python37\lib\site-packages\urllib3\_collections.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, MutableMapping
C:\Users\yujun\AppData\Local\Programs\Python\Python37\lib\site-packages\_distutils_hack\__init__.py:19: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  "Distutils was imported before Setuptools. This usage is discouraged "
[2021-08-23 16:02:59,64

In [2]:
def convert_example(example, tokenizer, max_seq_length=512, is_test=False):
    if not is_test:
        label_text = id2label[example["labels"]]
        target = tokenizer(label_text,return_token_type_ids=False,return_attention_mask=True)

    if (int(is_test) + len(example)) == 2:
        source = tokenizer(example["sentence"], max_seq_len=max_seq_length,return_attention_mask=True)
    else:
        source = tokenizer(
            example["sentence1"],
            text_pair=example["sentence2"],
            max_seq_len=max_seq_length,
            return_token_type_ids=False,
            return_attention_mask=True
        )

    if not is_test:
        return source["input_ids"],source["attention_mask"], target["input_ids"], target["attention_mask"]
    else:
        return source["input_ids"],source["attention_mask"]

trans_func = partial(
    convert_example,
    tokenizer=pd_tokenizer,
    max_seq_length=128,
    is_test=False
)
train_ds = train_ds.map(trans_func, lazy=True)

In [4]:
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=pd_tokenizer.pad_token_id),  # input_ids
    Pad(axis=0, pad_val=pd_tokenizer.pad_token_id),  # attention_mask
    Pad(axis=0, pad_val=-100),  # lm_labels
    Pad(axis=0, pad_val=pd_tokenizer.pad_token_id), # decoder_attention_mask
): fn(samples)
train_data_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    collate_fn=batchify_fn,
    num_workers=0,
    shuffle=True,
    return_list=True,
)

In [5]:
for e in train_data_loader:
    print(e)
    break

[Tensor(shape=[2, 58], dtype=int32, place=CPUPlace, stop_gradient=True,
       [[438  , 2386 , 12037, 53   , 34   , 429  , 1835 , 35   , 165  , 3429 , 3    , 8389 , 12   , 1733 , 8    , 3654 , 4029 , 6    , 420  , 837  , 14261, 32   , 17   , 23   , 1016 , 6    , 14702, 3588 , 6    , 243  , 8    , 192  , 4458 , 906  , 12   , 1132 , 8    , 1419 , 5    , 1    , 14702, 3588 , 19   , 3    , 9    , 5237 , 13   , 3    , 9    , 420  , 837  , 14261, 32   , 17   , 23   , 1016 , 5    , 1    ],
        [16908, 15507, 348  , 6    , 3    , 9    , 973  , 5812 , 44   , 5847 , 31   , 7    , 14914, 1015 , 636  , 6    , 65   , 3    , 8689 , 16   , 2405 , 3535 , 973  , 11   , 1284 , 973  , 21   , 72   , 145  , 460  , 203  , 5    , 1    , 16908, 15507, 348  , 19   , 3    , 9    , 973  , 5812 , 5    , 1    , 0    , 0    , 0    , 0    , 0    , 0    , 0    , 0    , 0    , 0    , 0    , 0    , 0    , 0    , 0    ]]), Tensor(shape=[2, 58], dtype=int32, place=CPUPlace, stop_gradient=True,
       [[1, 1, 1, 1, 1,

In [6]:
from transformers.models.t5 import T5ForConditionalGeneration as PTT5ForConditionalGeneration
PREFIX = "E:/paddle论文复现/suoyoudaima/paddle_t5-old/"
pt_model = PTT5ForConditionalGeneration.from_pretrained(PREFIX+"google/t5-small")
pt_model.eval()
print("=")

=


In [18]:
import torch
e = iter(train_data_loader).next()
x0 = torch.tensor(e[0].numpy())
x1 = torch.tensor(e[1].numpy())

In [8]:
from paddlenlp.transformers import T5ForConditionalGeneration as PDT5ForConditionalGeneration
pd_model = PDT5ForConditionalGeneration.from_pretrained(PREFIX+"paddle/t5-small")
pd_model.eval()
print("=")

=


In [9]:
def compare(a, b):
    a = torch.tensor(a.numpy()).float()
    b = torch.tensor(b.numpy()).float()
    meandif = (a - b).abs().mean()
    maxdif = (a - b).abs().max()
    print("mean difference:", meandif)
    print("max difference:", maxdif)

In [19]:
pd_outs = pd_model.generate(input_ids=e[0],attention_mask=e[1],num_beams=4,min_length=4,max_length=50-1,decode_strategy="beam_search",early_stopping=True,length_penalty=2.0)


In [20]:
pt_outs = pt_model.generate(input_ids=x0,attention_mask=x1,num_beams=4,min_length=4,max_length=50,early_stopping=True,length_penalty=2.0)

is_beam_gen_modeis_beam_gen_modeis_beam_gen_modeis_beam_gen_mode


In [21]:
compare(pt_outs,pd_outs[0])

mean difference: tensor(0.)
max difference: tensor(0.)
